In [38]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Remplacez "CLIENT_ID" et "CLIENT_SECRET" par vos propres identifiants de l'application Spotify
client_id = "0aa04daff5e64a329994247191f4634c"
client_secret = "1b5fa37daf72468e8c657afb010e613e"

# Créez une instance de l'authentification client-credentials pour utiliser l'API Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Recherchez des morceaux correspondant à une requête spécifique
results = sp.search(q='artist:Ed Sheeran track:Shape of You', type='track')

# Affichez le nom de chaque morceau correspondant à la requête
for track in results['tracks']['items']:
    print(track['name'])

Shape of You
Shape of You
Shape of You - Acoustic
Shape of You (feat. Zion & Lennox) - Latin Remix
Shape of You (feat. Nyla & Kranium) - Major Lazer Remix
Shape of You - Galantis Remix
Shape of You - Stormzy Remix
Shape of You - NOTD Remix
Shape of You - Yxng Bane Remix
Shape of You


In [39]:

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

track_id = 'spotify:track:6rqhFgbbKwnb9MLmUQDhG6' # Example track ID
track_features = sp.audio_features(track_id)
print(track_features)

[{'danceability': 0.592, 'energy': 0.0196, 'key': 1, 'loudness': -33.35, 'mode': 1, 'speechiness': 0.0358, 'acousticness': 0.362, 'instrumentalness': 0.854, 'liveness': 0.108, 'valence': 0.0312, 'tempo': 119.504, 'type': 'audio_features', 'id': '6rqhFgbbKwnb9MLmUQDhG6', 'uri': 'spotify:track:6rqhFgbbKwnb9MLmUQDhG6', 'track_href': 'https://api.spotify.com/v1/tracks/6rqhFgbbKwnb9MLmUQDhG6', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6rqhFgbbKwnb9MLmUQDhG6', 'duration_ms': 65314, 'time_signature': 4}]


In [40]:
import base64
import requests


client_ID = "0aa04daff5e64a329994247191f4634c"
client_secret = "1b5fa37daf72468e8c657afb010e613e"

# encode client_id and client_secret in base64
client_credentials = f"{client_id}:{client_secret}".encode("ascii")
base64_credentials = base64.b64encode(client_credentials).decode("ascii")

# make request for access token
url = "https://accounts.spotify.com/api/token"
response = requests.post(
    url,
    headers={
        "Authorization": f"Basic {base64_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    },
    data={"grant_type": "client_credentials"}
)

if response.status_code == 200:
    access_token = response.json()["access_token"]
    print(f"Access Token: {access_token}")
else:
    print(f"Error: {response.text}")

Access Token: BQAk0qQ_8zgbPN3GkO0K0V7SNwudm_IcXrKpSPapI8WtlKq6LdsxMgZ_Rq27OmdyEYk-RIwIYdT4BWa56ikNi3QpzIQ8brRZJP8sQ8UzSZqic3Khxbf9


In [45]:
import pandas as pd
import requests
from tqdm import tqdm

client_id = "0aa04daff5e64a329994247191f4634c"
client_secret = "1b5fa37daf72468e8c657afb010e613e"


# Obtenez les caractéristiques musicales et la popularité de chaque piste
def get_audio_features(track_id):
    global access_token
    url = f"https://api.spotify.com/v1/audio-features/{track_id}"
    response = requests.get(
        url,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
    )
    if response.status_code == 200:
        data = response.json()
        return [
            data["danceability"],
            data["energy"],
            data["key"],
            data["loudness"],
            data["mode"],
            data["speechiness"],
            data["acousticness"],
            data["instrumentalness"],
            data["liveness"],
            data["valence"],
            data["tempo"],
            data["duration_ms"]
        ]
    else:
        return None

# Requêtez l'API Spotify pour obtenir les pistes sorties en 2022
def get_new_tracks():
    global access_token
    tracks = []
    url = "https://api.spotify.com/v1/search"
    params = {
        "q": "year:2022",
        "type": "track",
        "market": "US",
        "limit": 50,
        "offset": 0
    }
    while True:
        response = requests.get(
            url,
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {access_token}"
            },
            params=params
        )
        if response.status_code == 200:
            data = response.json()
            tracks += data["tracks"]["items"]
            if data["tracks"]["next"] is None:
                break
            params["offset"] += 50
        else:
            print("Erreur lors de la récupération des pistes : ", response.text)
            break
    return tracks

# Récupérez les pistes et leurs caractéristiques musicales
def get_tracks_audio_features():
    global access_token
    tracks = get_new_tracks()
    audio_features = []
    for track in tqdm(tracks):
        track_info = {
            "id": track["id"],
            "name": track["name"],
            "album": track["album"]["name"],
            "artists": [artist["name"] for artist in track["artists"]],
            "popularity": track["popularity"],
            "explicit": track["explicit"],
            "release_date": track["album"]["release_date"],
            "duration_ms": track["duration_ms"]
        }
        features = get_audio_features(track["id"])
        if features is not None:
            track_info.update({
                "acousticness": features[6],
                "danceability": features[0],
                "energy": features[1],
                "instrumentalness": features[7],
                "key": features[2],
                "liveness": features[8],
                "loudness": features[3],
                "mode": features[4],
                "speechiness": features[5],
                "tempo": features[10],
                "valence": features[9]
            })
            audio_features.append(track_info)
    return pd.DataFrame(audio_features)

df = get_tracks_audio_features()
print(df.head())

100%|██████████| 1000/1000 [01:07<00:00, 14.80it/s]

                       id                                    name  \
0  3OHfY25tqY28d16oZczHc8                               Kill Bill   
1  2dHHgzDwk4BJdRwy9uXhTO  Creepin' (with The Weeknd & 21 Savage)   
2  1Qrg8KqiBpW07V7PNxwwwL                               Kill Bill   
3  1bDbXMyjaUIooNwFE9wn0N                               Rich Flex   
4  4FyesJzVpA39hbYvcseO2d                         Just Wanna Rock   

               album                                artists  popularity  \
0                SOS                                  [SZA]          92   
1  HEROES & VILLAINS  [Metro Boomin, The Weeknd, 21 Savage]          93   
2                SOS                                  [SZA]          92   
3           Her Loss                     [Drake, 21 Savage]          89   
4    Just Wanna Rock                         [Lil Uzi Vert]          88   

   explicit release_date  duration_ms  acousticness  danceability  energy  \
0     False   2022-12-09       153946        0.0543      

In [43]:
import pandas as pd
import numpy as np


# Calculate some statistics on the data
mean_liveness = np.mean(df['liveness'])
max_duration = np.max(df['duration_ms'])

print('Mean liveness: ', mean_liveness)
print('Max duration: ', max_duration)

KeyError: 'liveness'

In [1]:
df.columns

NameError: name 'df' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Define X and y
X = df.drop(['popularity'], axis=1)
y = df['popularity']

# Convert the list column to string column
X['artists'] = X['artists'].apply(lambda x: ','.join(map(str, x)))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Preprocess the training data
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_cols),
                                                ('cat', cat_transformer, cat_cols)])

X_train = preprocessor.fit_transform(X_train)

# Train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Preprocess the test data
X_test['artists'] = X_test['artists'].apply(lambda x: ','.join(map(str, x)))
X_test = preprocessor.transform(X_test)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

MAE: 4.578561680055512


NameError: name 'df' is not defined

In [ ]:
import mlflow
import mlflow.sklearn

# Assume you have already trained and evaluated a model
with mlflow.start_run() as run:
    # Log the model parameters and performance metrics
    mlflow.log_param('model_type', 'linear_regression')
    mlflow.log_param('num_features', X.shape[1])
    mlflow.log_metric('MAE', mae)

    # Log the trained model as an artifact
    mlflow.sklearn.log_model(model, 'model')
